In [167]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [168]:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [169]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [170]:
# Get a fun fact
response = ollama.chat.completions.create(model="llama3.2", messages=[{"role":"user","content":"Tell me a fun fact"}])
response.choices[0].message.content

'Did you know that there is a species of jellyfish that is immortal?! The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and start its life cycle all over again!'

In [171]:
from scraper import fetch_website_contents
from IPython.display import Markdown, display

In [172]:
# A simple QnA
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

'The answer is 4.'

In [173]:
# Fetching website contents
ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

In [174]:
# Define our system prompt 
system_prompt = """
You are a snarky assistant that analyzes the contents of website,
and provides a short, snarky humorous summary, ignoring text that might be navigation related.
Respons in markdown. Do not wrap the markdown in a code block - respond just with the markdown."""

In [175]:
# Define our user prompt
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or annoucements, then summarize these too. 
"""

In [176]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [177]:
messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of website,\nand provides a short, snarky humorous summary, ignoring text that might be navigation related.\nRespons in markdown. Do not wrap the markdown in a code block - respond just with the markdown.'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or annoucements, then summarize these too. \nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re 

In [178]:
# Summarize the website contents
def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [179]:
summarize("https://edwarddonner.com")

"## Summary: A Website by Edward Donner\n\nEdward Donner appears to be a former AI entrepreneur and CTO, currently running Nebula.io. The website showcases his personal interests, including writing code, DJing, and geeking out over LLMs on Hacker News. It also highlights his professional achievements in AI, including founding and CEO-ing untapt, which was acquired in 2021.\n\n## News and Announcements\n\n* On November 11, 2025: The Unique Energy of an AI Live Event\n* On September 15, 2025: AI in Production: Gen AI and Agentic AI on AWS at scale\n* On May 28, 2025: Be an AI Engineer and Leader: The Curriculum\n* In 2025, Edward Donner's organization published the 2025 AI Executive Briefing ( specifics not mentioned)\n\nNote that news and events are sparse; it mostly seems to be promotional or announcement-focused."

In [180]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [181]:
display_summary("https://edwarddonner.com")

* Meet Ed, a AI enthusiast who spends his time DJing, making music, and trying to keep up with the latest AI advancements in talent acquisition. The 2025 AI Executive Briefing is a must-read for any interested party on what's hot in AI talent management, including patented matching models and a healthy dose of proprietary LLMs.
* Upcoming events:
	+ September 15: "AI in Production: Gen AI and Agentic AI on AWS at scale"
	+ May 28: "Be an AI Engineer and Leader: The Curriculum" 
	+ May 18: AI Executive Briefing

In [182]:
display_summary("https://cnn.com")

**Summary**

This website appears to be the homepage of CNN (Cable News Network), a 24-hour news channel providing breaking news, live updates, and analysis on various topics.

# **News and Announcements**

* The current "Latest News" section has headlines and brief summaries.
	+ Breaking News: Ukraine-Russia War, Israel-Hamas War
    + Business: Latest market trends, US Politics
    Examples:
    * "Israel launches airstrikes in Gaza Strip amid escalation with Hamas"
    * "US stock market falls as inflation concerns rise"

Note there is a good mix of topic headings.

In [183]:
display_summary("https://anthropic.com")

# Marketing Speak Alert!
Anthropic is a fancy-schmancy company trying to save the world with its "public benefit corporation" act. It publishes some info about its "safety at the frontier" research, which sounds like corporate jargon for "we want to make sure AI doesn't destroy us all." 

# News in a Nutshell
The company is thrilled to announce Claude Opus 4.5, apparently the "best model in the world for coding, agents, computer use, and enterprise workflows." Read more about this revolutionary tool and what it means exactly. They're also talking a bit about AI's impact on society and how they want to use their fancy tech to ensure humanity's long-term well-being.

And let's not forget the most exciting part: people are saying things like "what exactly is it that you’re talking to?" when conversing with AI models. How original, right?

In [184]:
# Tokenizing with code

In [185]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
tokens = encoding.encode("Hi my name is Rujjul and I like Chocolate pastry.")


In [186]:
tokens

[13347, 856, 836, 374, 432, 9832, 73, 360, 323, 358, 1093, 39520, 74155, 13]

In [187]:
for token_id in tokens:
    token_text = encoding.decode([token_id])
    print(f"{token_id} = {token_text}")

13347 = Hi
856 =  my
836 =  name
374 =  is
432 =  R
9832 = uj
73 = j
360 = ul
323 =  and
358 =  I
1093 =  like
39520 =  Chocolate
74155 =  pastry
13 = .


In [188]:
encoding.decode([2911])

' children'

In [189]:
# The Illusion of "memory"

In [190]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi ! I am Rujjul"}
]

In [191]:
response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

"Hey Rujjul! It's nice to meet you. I hope you're having a great day so far. Is there something I can help you with, or would you like to chat for a bit?"

In [192]:
# ok, let's ask a follow-up question

In [193]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "what's my name?"}
]

In [194]:
response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

"I don't have any information about your identity, so I'm not sure what your name is. Would you like to tell me?"

In [195]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Rujjul!"},
    {"role": "assistant", "content": "Hi Rujjul! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]

In [196]:
response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

'Your name is Rujjul.'

In [197]:
# Creating a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits. 

In [198]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [199]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API Key looks good so far")
else:
    print("There might be a problem with your API Key? Please visit the troubleshooting notebook!")    

MODEL = 'llama3.2'
openai = OpenAI()

API Key looks good so far


In [200]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

In [201]:
link_system_prompt = """"
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links":[
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [202]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [203]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [204]:
def select_relevant_links(url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [205]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/', 'relativeUrl': ''},
  {'type': 'blog posts page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'connect-four game page',
   'url': 'https://edwarddonner.com/connect-four/',
   'relativeUrl': ''},
  {'type': 'outsmart game page',
   'url': 'https://edwarddonner.com/outsmart/',
   'relativeUrl': ''}]}

In [206]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'About company page', 'url': 'https://huggingface.co/'},
  {'type': 'Models page', 'url': 'https://huggingface.co/models'},
  {'type': 'Datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'Spaces/HunyuanVideo-1.5',
   'url': 'https://tencent.HunyuanVideo-1.5/huggingface.co/spaces/models'},
  {'type': 'Spaces/FLUX.2-dev',
   'url': 'https://facebook/sam3.huggingface.co/spaces/black-forest-labs/FLUX.2-dev'},
  {'type': 'Facebook Sam3 space',
   'url': 'https://facebook.sam3.huggingface.co '},
  {'type': 'Tencent Hunyuan Video Page',
   'url': 'https://tencent.HunyuanVideo-1.5/huggingface.co/en'},
  {'type': 'Black Forest Labs FLUX.2-dev page',
   'url': 'http://black-forest-labs.huggingface.co/FLUX.2-dev '},
  {'type': 'Tongyi-MAI Z-Image-Turbo Space',
   'url': 'https://tencent.HunyuanVideo-1.5/huggingface.co/spaces/Tongyi-MAI/Z-Image-Turbo'},
  {'type': 'Linoyts Qwen Image Edit Angles page',
   'url': 'https://facebook.sam3.huggingface.co/spaces/li

In [207]:
# Make a brochure

In [208]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [209]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
1 day ago
•
2.12k
•
727
black-forest-labs/FLUX.2-dev
Updated
9 minutes ago
•
116k
•
601
facebook/sam3
Updated
7 days ago
•
182k
•
723
Tongyi-MAI/Z-Image-Turbo
Updated
about 5 hours ago
•
2.07k
•
488
tencent/HunyuanOCR
Updated
about 21 hours ago
•
24.9k
•
443
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.3k
Qwen Image Edit Camera Control
🎬
1.3k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
Zero
181
FLUX.2 [dev]
💻
181
Generate detailed images from text prompts or edit existing images
Running
on
Zero
172
Z Image Turbo
🏃
172
Generate images from text prompts
Running
on
Zero
MCP
